In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet101, VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

header1

In [2]:
# Assuming the dataset is structured with subdirectories for each class (e.g., 'Trachoma' and 'Healthy')
data_dir = r'C:\Users\Habeeb\Downloads\Finale\dataset'

# ImageDataGenerator for preprocessing and data augmentation
datagen = ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.2,  # 20% of the data for validation
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
    
)

# Training data generator
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size = (224, 224),  # Resize images to 224x224
    batch_size = 32,
    class_mode = 'binary',
    subset = 'training'  # Use this subset for training
)

# Validation data generator
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size = (224, 224),
    batch_size = 32,
    class_mode = 'binary',
    subset = 'validation'  # Use this subset for validation
)

Found 1325 images belonging to 2 classes.
Found 331 images belonging to 2 classes.


#Build CNN model ResNet101 and VGG16

In [3]:
# Callbacks
checkpoint_resnet = ModelCheckpoint("resnet101_best_model.keras", monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
checkpoint_vgg16 = ModelCheckpoint("vgg16_best_model.keras", monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

# ResNet101
def create_resnet101_model(input_shape):
    base_model = ResNet101(weights='imagenet', include_top=False, input_shape=input_shape)
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    return model

input_shape = (224, 224, 3)
resnet101_model = create_resnet101_model(input_shape)

In [4]:
# VGG16
def create_vgg16_model(input_shape):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    model = Sequential([
        base_model,
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    return model

vgg16_model = create_vgg16_model(input_shape)

In [ ]:
# Training the ResNet101 model
history_resnet101 = resnet101_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=50,  # Increase the number of epochs
    callbacks=[checkpoint_resnet, early_stopping]
)


Epoch 1/50


C:\Users\Habeeb\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 8/41 ━━━━━━━━━━━━━━━━━━━━ 17:01 31s/step - accuracy: 0.4880 - loss: 0.8114

In [ ]:
# Training the VGG16 model
history_vgg16 = vgg16_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=50,  # Increase the number of epochs
    callbacks=[checkpoint_vgg16, early_stopping]
)


In [ ]:
# Evaluate the ResNet101 model
val_loss_resnet101, val_accuracy_resnet101 = resnet101_model.evaluate(validation_generator)
print(f'ResNet101 Validation Loss: {val_loss_resnet101}')
print(f'ResNet101 Validation Accuracy: {val_accuracy_resnet101}')

# Generate classification report and confusion matrix
val_predictions_resnet101 = resnet101_model.predict(validation_generator)
val_predictions_resnet101 = (val_predictions_resnet101 > 0.5).astype(int)  # Convert probabilities to binary output

val_labels = validation_generator.classes
class_labels = list(validation_generator.class_indices.keys())

In [ ]:
# Evaluate the VGG16 model

val_loss_vgg16, val_accuracy_vgg16 = vgg16_model.evaluate(validation_generator)
print(f'VGG16 Validation Loss: {val_loss_vgg16}')
print(f'VGG16 Validation Accuracy: {val_accuracy_vgg16}')

# Generate classification report and confusion matrix
val_predictions_vgg16 = vgg16_model.predict(validation_generator)
val_predictions_vgg16 = (val_predictions_vgg16 > 0.5).astype(int)  # Convert probabilities to binary output

# Classification report
print(classification_report(val_labels, val_predictions_vgg16, target_names=class_labels))

# Confusion matrix
conf_matrix_vgg16 = confusion_matrix(val_labels, val_predictions_vgg16)
print(conf_matrix_vgg16)

In [ ]:
# Classification report
print(classification_report(val_labels, val_predictions_resnet101, target_names=class_labels))

# Confusion matrix
conf_matrix_resnet101 = confusion_matrix(val_labels, val_predictions_resnet101)
print(conf_matrix_resnet101)

In [ ]:
# Evaluate the VGG16 model
val_loss_vgg16, val_accuracy_vgg16 = vgg16_model.evaluate(validation_generator)
print(f'VGG16 Validation Loss: {val_loss_vgg16}')
print(f'VGG16 Validation Accuracy: {val_accuracy_vgg16}')

In [ ]:
# Generate classification report and confusion matrix
val_predictions_vgg16 = vgg16_model.predict(validation_generator)
val_predictions_vgg16 = (val_predictions_vgg16 > 0.5).astype(int)  # Convert probabilities to binary output

# Classification report
print(classification_report(val_labels, val_predictions_vgg16, target_names=class_labels))


In [ ]:
# Confusion matrix
conf_matrix_vgg16 = confusion_matrix(val_labels, val_predictions_vgg16)
print(conf_matrix_resnet101)

# Plot confusion matrix for ResNet101
plt.matshow(conf_matrix_resnet101, cmap='coolwarm')
plt.title('ResNet101 Confusion Matrix')
plt.colorbar()
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig('resnet101.png')
plt.show()

In [ ]:
# Plot confusion matrix for VGG16
plt.matshow(conf_matrix_vgg16, cmap='coolwarm')
plt.title('VGG16 Confusion Matrix')
plt.colorbar()
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig('vgg16_confusion_matrix.png')  # Save the figure
plt.show()

In [ ]:
# Plot training & validation accuracy values for ResNet101
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history_resnet101.history['accuracy'])
plt.plot(history_resnet101.history['val_accuracy'])
plt.title('ResNet101 Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.savefig('resnet101_accuracy.png')

In [ ]:
# Plot training & validation loss values for ResNet101
plt.subplot(1, 2, 2)
plt.plot(history_resnet101.history['loss'])
plt.plot(history_resnet101.history['val_loss'])
plt.title('ResNet101 Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.savefig('resnet101_loss.png') 
plt.show()

In [ ]:
# Plot confusion matrix for VGG16
plt.matshow(conf_matrix_vgg16, cmap='coolwarm')
plt.title('VGG16 Confusion Matrix')
plt.colorbar()
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig('vgg16_confusion_matrix.png')  # Save the figure
plt.show()

In [ ]:
# Plot training & validation accuracy values for VGG16
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history_vgg16.history['accuracy'])
plt.plot(history_vgg16.history['val_accuracy'])
plt.title('VGG16 Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.savefig('vgg16_accuracy.png')  

In [ ]:
# Plot training & validation loss values for VGG16
plt.subplot(1, 2, 2)
plt.plot(history_vgg16.history['loss'])
plt.plot(history_vgg16.history['val_loss'])
plt.title('VGG16 Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.savefig('vgg16_loss.png')
plt.show()

In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score


# Get true labels
true_labels = validation_generator.classes

# Get predicted probabilities
# For binary classification, use the probability of the positive class (class 1)
predicted_probabilities_resnet101 = resnet101_model.predict(validation_generator)
predicted_probabilities_vgg16 = vgg16_model.predict(validation_generator)

# Calculate ROC AUC score
roc_auc_resnet101 = roc_auc_score(true_labels, predicted_probabilities_resnet101)
roc_auc_vgg16 = roc_auc_score(true_labels, predicted_probabilities_vgg16)

print(f'ROC AUC for ResNet101: {roc_auc_resnet101}')
print(f'ROC AUC for VGG16: {roc_auc_vgg16}')